In [ ]:
import pandas as pd
from matplotlib import pyplot as plt
from long_covid.colors import flatuicolors
from long_covid import styling
import numpy as np
import matplotlib.dates as mdates

In [ ]:
TESTDATA = '../data/raw/tests.feather'
VITALS = '../data/raw/vitals_processed.feather'

BASELINE_LENGTH = 54

In [ ]:
tests = pd.read_feather(TESTDATA)
vitals = pd.read_feather(VITALS)

In [ ]:
def plot_example_timeseries(vitals, tests, unvacc_user, unvacc_user_device, vacc_user, vacc_user_device, filename):
    
    f, axarr = plt.subplots(3, 2, sharex='col', figsize=(8, 4))
    
    left_panel = (unvacc_user, unvacc_user_device, flatuicolors.pomegranate, axarr[:, 0])
    right_panel = (vacc_user, vacc_user_device, flatuicolors.wisteria, axarr[:, 1])
    
    plot_data = {
        65: {'label': 'Resting heart rate\n[bpm]', 'ymin': None, 'ymax': None, 'ylim': None, 'norm': 1},
        9: {'label': 'Activity\n[steps/day]', 'ymin': None, 'ymax': 20000, 'ylim': (0, 15000), 'norm': 1 },
        43: {'label': 'Sleep duration\n[hrs/day]','ymin': 250, 'ymax': None, 'ylim': (5, 15), 'norm': 60 }
    }
    
    for userid, deviceid, color, axes in (left_panel, right_panel):
        
        testdate = tests[tests.user_id == userid].test_date.values[0]

        for ax, vitalid in zip(axes, (65, 9, 43)):

            to_plot = vitals[
                (vitals.userid == userid) & 
                (vitals.vitalid == vitalid) & 
                (vitals.deviceid == deviceid) &
                (vitals.date.between(testdate - np.timedelta64(BASELINE_LENGTH,'D'), testdate + np.timedelta64(90,'D')))
            ]
            
            x, y = to_plot[['date', 'raw_value']].sort_values(by='date').values.T

            if plot_data[vitalid]['ymin']:
                x = x[y > plot_data[vitalid]['ymin']]
                y = y[y > plot_data[vitalid]['ymin']]
                
            if plot_data[vitalid]['ymax']:
                x = x[y < plot_data[vitalid]['ymax']]
                y = y[y < plot_data[vitalid]['ymax']]
              
            if plot_data[vitalid]['ylim']:
                ax.set_ylim(plot_data[vitalid]['ylim'])
                
            y = y / plot_data[vitalid]['norm']
            ax.set_ylabel(plot_data[vitalid]['label'])
                
            ax.plot(x, y, c=color, lw=1.2, alpha=0.9, zorder=10)
            ax.axhline(y[:BASELINE_LENGTH].mean(), color=flatuicolors.midnightblue, ls='--')
            
        rect = plt.Rectangle((testdate, -0.04), width=np.timedelta64(7,'D'), height=3+2*f.subplotpars.hspace+0.15,
                     transform=ax.get_xaxis_transform(), clip_on=False,
                     edgecolor='none', facecolor="k", alpha=0.25, linewidth=3)
        
        ax.add_patch(rect)
        ax.xaxis.set_major_formatter(mdates.ConciseDateFormatter(ax.xaxis.get_major_locator()))

    for ax in axarr[:2, 0]:
        styling.hide_and_move_axis(ax, hide=['top', 'right', 'bottom'])
        ax.xaxis.set_visible(False)
        
    for ax in axarr[:2, 1]:
        styling.hide_and_move_axis(ax, hide=['top', 'left', 'bottom'])
        ax.xaxis.set_visible(False)
        
    styling.hide_and_move_axis(axarr[2, 0], hide=['top', 'right'])
    styling.hide_and_move_axis(axarr[2, 1], hide=['top', 'left'])

    axarr[0, 0].set_ylim(60, 80)
    axarr[0, 1].set_ylim(45, 60)

    plt.subplots_adjust(wspace=0.03, hspace=0.24, bottom=0.075, top=0.97, left=0.14, right=0.86)
    plt.savefig(filename, dpi=400)

In [ ]:
plot_example_timeseries(vitals, tests, 276043, 2, 460177, 2, '../output/timeseries_processed.jpg')